In [ ]:
import JSON
using Plots
using Trapz
using HDF5
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/disturbances.jl")

In [1]:
border = import_border("../data/border.json")
dx = 20
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))
x = repeat(reshape(xrange,1,Nx),Ny,1)
y = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_bx = repeat(reshape(xrange,1,Nx),Ny,1) .- dx/2
y_for_bx = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_by = repeat(reshape(xrange,1,Nx),Ny,1)
y_for_by = repeat(reshape(yrange,Ny,1),1,Nx) .+ dx/2;

LoadError: UndefVarError: import_border not defined

In [ ]:
sigma = 1E2
recompute_param = false

if(recompute_param)
    bx, by, p, m, d = get_params(isborder, sigma,
        "../data/pantagruel.h5", "../numerics/grid_params_" * string(dx) * ".h5")
else
    bx, by, p, m, d = get_params("../numerics/grid_params_" * string(dx) * ".h5")
end

#dP = -9.0
dP = 0.0
sigma = 30.0
location = [-1500., -900.]
dp = local_disturbance(isgrid, xrange, yrange, location, dP, sigma)
do_plot(isgrid, dp)

# u*dt^2/dx^2 should be << 1
u = zeros(Ny,Nx)
for j=2:Nx-1
    for i=2:Ny-1
        u[i,j] = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1]) / m[i,j]
    end
end
u[.!isgrid] .= 0;


In [ ]:
interval = 50
dt = 0.0001
Ndt = 5000

println(maximum(u)*dt^2/dx^2)

gamma = d ./ m

omegas = zeros(Ny,Nx,1 + Int64(ceil(Ndt/interval)))
thetas = zeros(Ny,Nx,1 + Int64(ceil(Ndt/interval)))
th_new = zeros(Ny,Nx)
reset = true
data = h5read("../numerics/stable_" * string(dx) * ".h5", "/")
if(reset)
    #th_old = zeros(Ny,Nx)
    #th = zeros(Ny,Nx)
    th_old = data["th"]
    th = data["th"]
    omegas[:,:,1] = th_old
end
println(maximum(abs.(th_old[isgrid]))) 
ts = zeros(1 + Int64(ceil(Ndt/interval)))
chi = 1 .+ gamma*dt/2

@time begin
    for k in 1:Ndt
        for i in 2:Ny-1
            for j in 2:Nx-1
                if(isinside[i,j])
                    bij = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1])
                    th_new[i,j] = (2 - bij / m[i,j] * dt^2 / dx^2) / chi[i,j] * th[i,j] + 
                        (gamma[i,j] * dt/2 - 1) /chi[i,j] * th_old[i,j] + 
                        dt^2 / dx^2 / chi[i,j] / m[i,j] * 
                        (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] +
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1]) +
                        dt^2 / chi[i,j] / m[i,j] * (p[i,j] + dp[i,j])
                end
            end
        end   
        
        # impose boundary condition 
        for k in 1:size(n,1)
            i = n[:,1]
            j = n[:,2]
            ny = n[id,3][1]
            nx = n[id,4][1]
            th_new[i,j] = (nx * ( bx[i,j] * th_new[i,j-1] - bx[i,j+1] * th_new[i,j+1]) +
                ny * (by[i-1,j] * th_new[i-1,j] - by[i,j] * th_new[i+1,j])) /
                (nx * (bx[i,j] - bx[i,j+1]) + ny * (by[i-1,j] - by[i,j]))
        end
        if(mod(k,interval) == 0)
            println("NIter: ", k)
            omegas[:,:,Int64(k/interval) + 1] = (th_new-th) / dt
            thetas[:,:,Int64(k/interval) + 1] = th_new
            ts[Int64(k/interval) + 1] = k*dt
        end
        th_old = copy(th)
        th = copy(th_new)
    end
end

In [ ]:
@gif for i=1:size(omegas,3)
    do_plot(isgrid, omegas[:,:,i])
end

In [ ]:
omegas[:,:,2]